In [1]:
## Import Library
import os
import copy
import json
import re
import logging
import datetime

import openai

#from langchain.llms import AzureOpenAI 
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.vectorstores.azuresearch import AzureSearch
#from langchain.chains.question_answering import load_qa_chain
#from langchain.retrievers import AzureCognitiveSearchRetriever

#from langchain.memory import ConversationBufferMemory, CosmosDBChatMessageHistory
from langchain.chains import (
    LLMChain, 
    #ConversationalRetrievalChain,
    # RetrievalQA,
    # SimpleSequentialChain,
    SequentialChain)
#from langchain.schema import HumanMessage, Document

# Import Azure OpenAI
# from azure.storage.blob import BlobServiceClient
# from azure.core.credentials import AzureKeyCredential
# from azure.identity import AzureDeveloperCliCredential
# from azure.search.documents import SearchClient
# from azure.search.documents.indexes import SearchIndexClient
# from azure.search.documents.indexes.models import (
#     HnswParameters,
#     PrioritizedFields,
#     SearchableField,
#     SearchField,
#     SearchFieldDataType,
#     SearchIndex,
#     SemanticConfiguration,
#     SemanticField,
#     SemanticSettings,
#     SimpleField,
#     VectorSearch,
#     VectorSearchAlgorithmConfiguration,
# )
# from azure.core.exceptions import ResourceExistsError

#from pypdf import PdfReader
from bing_search import bing_search_for_credit_proposal
#from langdetect import detect
from common import prompts as prompts


# Setting credit
INDEX_NAME = "credit-proposal"
SEARCH_SERVICE = "gptdemosearch"
SEARCH_API_KEY = "PcAZcXbX2hJsxMYExc2SnkMFO0D94p7Zw3Qzeu5WjYAzSeDMuR5O"
STORAGE_SERVICE = "creditproposal"
STORAGE_API_KEY = "hJ2qb//J1I1KmVeDHBpwEpnwluoJzm+b6puc5h7k+dnDSFQ0oxuh1qBz+qPB/ZT7gZvGufwRbUrN+ASto6JOCw=="
CONNECT_STR = f"DefaultEndpointsProtocol=https;AccountName={STORAGE_SERVICE};AccountKey={STORAGE_API_KEY}"
DOC_INTELL_ENDPOINT = "https://doc-intelligence-test.cognitiveservices.azure.com/"
DOC_INTELL_KEY = "9fac3bb92b3c4ef292c20df9641c7374"

OPENAI_API_BASE = ""
OPENAI_API_VERSION = ""
DEPLOYMENT_NAME = ""

# set up openai environment - Jay
OPENAI_API_TYPE = "azure"
OPENAI_API_BASE = "https://pwcjay.openai.azure.com/"
OPENAI_API_VERSION = "2023-09-01-preview"
OPENAI_API_KEY = "f282a661571f45a0bdfdcd295ac808e7"
DEPLOYMENT_NAME = "gpt-35-16k"

#set up openai environment - Ethan
# OPENAI_API_TYPE = "azure"
# OPENAI_API_BASE = "https://lwyethan-azure-openai-test-01.openai.azure.com/"
# OPENAI_API_VERSION = "2023-09-01-preview"
# OPENAI_API_KEY = "ad3708e3714d4a6b9a9613de82942a2b"
# DEPLOYMENT_NAME = "gpt-35-turbo-16k"

#set up openai environment - Cyrus
# OPENAI_API_TYPE = "azure"
# OPENAI_API_BASE = "https://pwc-cyrus-azure-openai.openai.azure.com/"
# OPENAI_API_KEY = "e1948635e8024556a6a55e37afcce932"
# DEPLOYMENT_NAME = "chat"

# set up openai environment - Sonia
#os.environ["OPENAI_API_TYPE"] = "azure"
#os.environ["OPENAI_API_BASE"] = "https://demo-poc-schung.openai.azure.com/"
#os.environ["OPENAI_API_VERSION"] = "2023-09-01-preview"
#os.environ["OPENAI_API_KEY"] = "c443f898db514f51822efd2af06154fc"
#DEPLOYMENT_NAME="demo-model-gpt4"

# Setting up ACS -Jay
#os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = SEARCH_SERVICE
#os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = SEARCH_API_KEY
#os.environ["AZURE_INDEX_NAME"] = INDEX_NAME

# set up openai environment
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

def cap(match):
    return(match.group().capitalize())

# load json data function
def load_json(json_path):
    with open(json_path, "r" ,encoding="utf-8") as f:
        data = json.load(f)
    return data



def clean_generated_text(text, client, section_name):
    #replacement
    text = text.replace("Based on the information provided, ", "").replace("Based on the given information, ", "").replace("It is mentioned that ", "").replace("...", ".").replace("..", ".")
    
    #reformat the client name
    insensitive_replace = re.compile(re.escape(client.lower()), re.IGNORECASE)
    text = insensitive_replace.sub(client, text)

    #Drop some unwanted sentences
    sentence_list = re.split(r"(?<=[.?!] )", text)
    unwanted_word_list = ["ABC ", 
                          "XYZ ", 
                          "GHI", 
                          "DEF ", 
                          "RM Notes do not provide", 
                          "RM Note does not provide", 
                          "does not provide specific details", 
                          "it is difficult to assess", 
                          "is not mentioned in the RM Notes", 
                          "not provided in the RM Notes", 
                          "not explicitly mentioned", 
                          "further information is needed", 
                          "no specific mention of", 
                          "not possible to provide ", 
                          "is not provided in the input information",
                          "is not provided",
                          "are not provided", 
                          "unable to extract ", 
                          "request further information",
                          "is no information available",
                          "are not available"]
    sentence_list_dropped = [sentence for sentence in sentence_list if all(word not in sentence for word in unwanted_word_list)]
    text = ' '.join(sentence_list_dropped)
    #Drop those sentence only = numbering point 
    out_sentence_list = []
    for l in text.split('\n'):
        if len(l) >= 2:
            if ((l[0].isdigit()) & ( l[1:].strip() == '.')) | (l.strip()[0:] == '-'):
                continue
        out_sentence_list.append(l)
    text = '\n'.join(out_sentence_list)
    #Remove the section name if it starts with it
    if len(text) >= len(section_name)+2:
        if text.lower().startswith(section_name.lower()+": "):
            text = text[len(section_name)+2:]
    #All capital letters for first letter in sentences
    formatter = re.compile(r'(?<=[\.\?!]\s)(\w+)')
    text = formatter.sub(cap, text)
    if len(text) >= 2:
        text = text[0].upper()+text[1:]
    return text.strip().replace("\n\n\n\n\n", "\n").replace("\n\n\n\n", "\n").replace("\n\n\n", "\n").replace("\n\n", "\n").replace(".  ", ". ").replace("!  ", "! ").replace("?  ", "? ").replace("in the RM Notes", "").replace('. . ', '. ').replace('. . . ', '. ').replace("[RM, ", "").replace("[", "").replace("]", "")



In [2]:
#This funcition is to prepare the rm note in desired format for web, call by app.py
def web_extract_RM(section, rm_note_txt, client, deployment_name=DEPLOYMENT_NAME, openai_api_version=OPENAI_API_VERSION, openai_api_base=OPENAI_API_BASE):
    hierarchy_file_name = "config/hierarchy_v3.json"

    hierarchy_dict_list = load_json(hierarchy_file_name)

    hierarchy_dict_list = hierarchy_dict_list["content"]

    prompt_template_for_extracting_rm_note = prompts.EXTRACTION_PROMPT


    print("#"*50)
    logging.info("rm_note_txt logging.info:", rm_note_txt)
    print("%"*50)
    print("rm_note_txt printing:", rm_note_txt)
    print("#"*50)
    rm_prompt_template = PromptTemplate(template=prompt_template_for_extracting_rm_note, input_variables=["rm_note", "question", "client_name", "example"])#"example",])

    # set up openai environment - Jay
    llm_rm_note = AzureChatOpenAI(deployment_name=deployment_name, temperature=0,
                            openai_api_version=openai_api_version, openai_api_base=openai_api_base, verbose=False)

    output_dict_list = []
    rm_text_list = []  # Variable to store the "[RM ...]" text

    for dictionary in hierarchy_dict_list:
        if dictionary["Section"] == section: #loop by section
            chain = LLMChain(llm=llm_rm_note, prompt=rm_prompt_template, verbose=False)
            dictionary["Value"] = chain({"rm_note":rm_note_txt, "question": dictionary["Question"], "client_name": client, "example": dictionary["Example"]})['text']
            # dictionary["Value"] = chain({"rm_note":rm_note_txt, "question": dictionary["Question"], "client_name": client,})['text']
            dictionary["Value"] = dictionary["Value"].replace("Based on the given information, ", "")
            # print("="*30)
            # print("dictionary[Value]:")
            # print(dictionary["Value"])
            # print("="*30)
            dictionary["Value"] = clean_generated_text(dictionary["Value"], client, section)
            # Use regular expressions to find the pattern "[RM ...]"
            for deleted_word in ["RM", "NA", "N/A"]:
                matches = re.findall(r"\[{} [^\]]+\]".format(deleted_word), dictionary["Value"], re.DOTALL)
                for match in matches:
                    dictionary["Value"] = dictionary["Value"].replace(match, "")
                    rm_text_list.append(match)
            output_dict_list.append(dictionary)
            print(output_dict_list)
            
    # Create Json file 
    # output_json_name = "GOGOVAN_hierarchy_rm_note.json"
    # json.dump(output_dict_list, open(output_json_name, "w"), indent=4)
    return output_dict_list, rm_text_list


def generate_rm_fill(rm_fill_values, client):
    # Remove the specific phrase "Please provide further information on" from each value in rm_fill_values
    # Then strip any leading/trailing whitespace and remove trailing periods
    rm_fill_values = [value.replace("Please provide further information on", "").strip().rstrip('.') for value in rm_fill_values]

    # Combine the RM_fill values into a single string separated by commas and "and" before the last value
    # Ensure that it doesn't end with a period or a comma
    if rm_fill_values:
        # Create a combined text of RM_fill values
        combined_rm_fill_text = ", ".join(rm_fill_values[:-1])
        if len(rm_fill_values) > 1:
            combined_rm_fill_text += ", and " + rm_fill_values[-1]
        else:
            combined_rm_fill_text = rm_fill_values[0]

        # Add the prefix "Please provide further information on" if it's not already present and appropriate
        if not combined_rm_fill_text.lower().startswith("please provide further information on"):
            combined_rm_fill_text = "Please provide further information on " + combined_rm_fill_text

        final_rm_fill_text = combined_rm_fill_text
    else:
        final_rm_fill_text = ""

    #reformat the client name
    insensitive_replace = re.compile(re.escape(client.lower()), re.IGNORECASE)
    final_rm_fill_text = insensitive_replace.sub(client, final_rm_fill_text)
    if (all(final_rm_fill_text.endswith(s) for s in ["!", "?", "."]) is False) & (len(final_rm_fill_text) > 0):
        final_rm_fill_text = final_rm_fill_text+'.'
    return final_rm_fill_text

In [3]:
# function to perform first generation of the paragraph
def first_generate(section_name, input_json, client, rm_text_variable, deployment_name=DEPLOYMENT_NAME, openai_api_version=OPENAI_API_VERSION, openai_api_base=OPENAI_API_BASE):
    """
    A core function to generate the proposal per section

    Parameters:
    -----------
    prompt: str
    Prompt text for instructing the output based on RM prompt

    rm_note: str
    It contains the information from the RM

    example: str
    Input example for GPt to take it as an example

    """
    # For each section, gen content based on its prompt.
    proposal_proposal_template_text = prompts.PROPOSAL_TEMPLATE_EXECUTIVE_SUMMARY if section_name == "Executive Summary" \
        else prompts.PROPOSAL_TEMPLATE_CLIENT_REQUEST if section_name == "Client Request" \
        else prompts.PROPOSAL_TEMPLATE_SHAREHOLDERS_AND_GROUP_STRUCTURE if section_name == "Shareholders and Group Structure" \
        else prompts.PROPOSAL_TEMPLATE_PROJECT_DETAILS if section_name == "Project Details" \
        else prompts.PROPOSAL_TEMPLATE_INDUSTRY_SECTION_ANALYSIS if section_name == "Industry / Section Analysis" \
        else prompts.PROPOSAL_TEMPLATE_MANAGEMENT if section_name == "Management" \
        else prompts.PROPOSAL_TEMPLATE_FINANCIAL_INFO_OF_BORROWER if section_name == "Financial Information of the Borrower" \
        else prompts.PROPOSAL_TEMPLATE_OTHER_BANKING_FACILITIES if section_name == "Other Banking Facilities" \
        else prompts.PROPOSAL_TEMPLATE_OPINION_OF_RELATIONSHIP_MANAGER if section_name == "Opinion of the Relationship Manager" \
        else prompts.PROPOSAL_TEMPLATE_SUMMARY_OF_RECOMMENDATION if section_name == "Summary of Recommendation" \
        else prompts.PROPOSAL_TEMPLATE_GENERIC

    # set temperature as 0 with exception cases
    temperature = 0.5 if section_name == "Industry / Section Analysis" else 0

    # set up openai environment - Jay
    llm_proposal = AzureChatOpenAI(deployment_name=deployment_name, temperature=temperature,
                            openai_api_version=openai_api_version, openai_api_base=openai_api_base,verbose=True)

    chain = LLMChain(
        llm=llm_proposal,
        prompt=PromptTemplate(template=proposal_proposal_template_text, input_variables=["input_info", "client_name", "example"]),
        output_key="first_gen_paragraph", verbose=True
    )

    review_chain = LLMChain(llm=llm_proposal
                            , prompt=PromptTemplate(template=prompts.PROPOSAL_TEMPLATE_REVIEW_PROMPT, input_variables=["first_gen_paragraph", "example"])
                            , output_key="reviewed",verbose=True)

    checking_formatting_chain = LLMChain(llm=llm_proposal
                                , prompt=PromptTemplate(template=prompts.PROPOSAL_TEMPLATE_FORMATTING_PROMPT, input_variables=["reviewed"])
                                , output_key="reviewed_2",verbose=True)


    if section_name in ["Industry / Section Analysis", "Summary of Recommendation"]:
        overall_chain = SequentialChain(chains=[chain], 
                                        input_variables=["input_info", "client_name", "example"],
                                        # Here we return multiple variables
                                        output_variables=["first_gen_paragraph"],
                                        verbose=True)
    else:
        overall_chain = SequentialChain(chains=[chain, review_chain, checking_formatting_chain, ], 
                                        input_variables=["input_info", "client_name", "example"],
                                        # Here we return multiple variables
                                        output_variables=["reviewed_2"],
                                        verbose=True)

    # Break the input_json by parts
    input_info_str = []
    example_str = []

    example_str_empty = False
    for item in input_json:
        sub_section = item['Sub-section']
        value = item['Value']
        example = item['Example']
        # Append sub_section and value only if value is not empty
        if value != "":  # This checks if value is not just whitespace
            input_info_str.append(f"{sub_section} : {value}")
        else:
            example_str_empty = True
        if example != "":
            example_str.append(f"{sub_section} : {example}")
    if example_str_empty:
        example_str = []

    # Call Bing Search if it's empty
    # Enter Bing Seach when the extract value is NULL
    disclaimer_of_bing_search = False
    if (len(rm_text_variable) > 0) | (section_name in ["Shareholders and Group Structure", "Management"]):
        #TODO: for GogoX only:
        bing_search_list, disclaimer_of_bing_search = bing_search_for_credit_proposal(
            client="GOGOX Holding Limited" if client.lower() == "gogox" else client
            , section_name=section_name) #will return a list
        input_info_str.extend(bing_search_list)
        

    final_dict = {"input_info": "\n\n".join(input_info_str), "Example": "\n\n".join(example_str)}
    print("="*50)
    print("="*50)
    print("\n>>> Section Name: ", section_name)
    print("\n>>> input_info:", final_dict["input_info"])
    print("+"*50)
    print("\n>>> example:", example_str)
    print("="*50)
    print("="*50)
    if len(input_info_str) > 0:
        drafted_text = overall_chain({"input_info": final_dict["input_info"], "client_name": client, "example": final_dict["Example"]})
        if section_name in ["Industry / Section Analysis", "Summary of Recommendation"]:
            drafted_text = drafted_text["first_gen_paragraph"]
            print(drafted_text)
        else:
            drafted_text = drafted_text["reviewed_2"]
    else:
        drafted_text = "There is no information for {} specified.".format(section_name)

    # Create blank list to store the "[RM Please provide ...]"
    # Remove the 'RM ' prefix and the brackets
    # Add the cleaned text to the rm_fill_values list
    rm_fill_values = [item.replace("RM ", "").strip("[]") for item in rm_text_variable]
    #print(rm_fill_values)

    lines = drafted_text.split("\n")

    for i, line in enumerate(lines):
        matches = re.findall(r"\[RM (.+?)\]\.?", line)  # Find all [RM ...] followed by optional dot
        for match in matches:
            rm_fill = match + "\n"
            rm_fill_values.append(rm_fill)
        
        # remove all the RM requests and the optional following dots from the line
        line = re.sub(r"\[RM .+?\]\.?", "", line)
        lines[i] = line

    # Rejoin the lines into a single string without RM requests
    drafted_text2 = "\n".join(lines)
    drafted_text3 = clean_generated_text(drafted_text2, client, section_name)
    final_rm_fill_text = generate_rm_fill(rm_fill_values, client)
    #print(rm_fill_values)

    #Edit the format of final rm fill
    if section_name == "Summary of Recommendation":
        if drafted_text3 == "Based on the RM notes, there is insufficient information to make a recommendation for the proposed loan facility. RM please provide your own judgement.":
            final_rm_fill_text = "RM please provide your own judgement."
        else:
            final_rm_fill_text = ""
    elif disclaimer_of_bing_search:
        disclaimer_of_bing_search_text = "The above generated content contains information from Bing Search, as there is missing information detected in the RM Note. Please help review and confirm it."
        final_rm_fill_text = disclaimer_of_bing_search_text + '\n' + final_rm_fill_text

    output_json = {
        "section": section_name,
        "output": drafted_text3,
        "RM_fill" : final_rm_fill_text,
    }

    #output the result
    return output_json


# Wrapper function
def run_first_gen(section, rm_note_txt, client, deployment_name=DEPLOYMENT_NAME, openai_api_version=OPENAI_API_VERSION, openai_api_base=OPENAI_API_BASE):
    extract_json, rm_text_variable = web_extract_RM(section, rm_note_txt, client
        , deployment_name=deployment_name, openai_api_version=openai_api_version, openai_api_base=openai_api_base)
    print("extract_json!!!!!!"*3)
    for l in extract_json:
        print(l['Sub-section']+":", l['Value'])
        print("="*30)
    print("extract_json!!!!!!"*3)
    #print("rm_text_variable!!!!!!"*3)
    for l in rm_text_variable:
        print(l)
        print("="*30)
    output_json = first_generate(section, extract_json, client, rm_text_variable
        , deployment_name=deployment_name, openai_api_version=openai_api_version, openai_api_base=openai_api_base)
    return output_json

In [4]:
#set up openai environment - Cyrus
OPENAI_API_TYPE = "azure"
OPENAI_API_BASE = "https://pwcjay.openai.azure.com/"
OPENAI_API_VERSION = "2023-09-01-preview"
OPENAI_API_KEY = "f282a661571f45a0bdfdcd295ac808e7"
DEPLOYMENT_NAME = "gpt-35-16k"

client = "GogoX"


#section_name ="Financial Information of the Borrower"

section_name = "Project Details"

RM_text="""

•Request $10 million to support its expansion plans        
•Repayment plan: Regular principal and interest payments over a 3 years term
 
•The Expansion plans include 3 areas:
•Expanding the business in 3 cities in China (Beijing, Shanghai and Shenzhen)
•Technology investments (AI assistant fo drivers and AI-based matching among drivers and customers)
•Working capital needs: expanding 300 more permanent staff, including customer service supports in the 3 new cities, technology experts in IT and AI.
 
•The timeline for this expansion is within the next 36 months.

"""

run_first_gen(section=section_name, 
              rm_note_txt=RM_text, 
              client=client, 
              deployment_name=DEPLOYMENT_NAME, 
              openai_api_version=OPENAI_API_VERSION, 
              openai_api_base=OPENAI_API_BASE)

##################################################
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
rm_note_txt printing: 

•Request $10 million to support its expansion plans        
•Repayment plan: Regular principal and interest payments over a 3 years term
 
•The Expansion plans include 3 areas:
•Expanding the business in 3 cities in China (Beijing, Shanghai and Shenzhen)
•Technology investments (AI assistant fo drivers and AI-based matching among drivers and customers)
•Working capital needs: expanding 300 more permanent staff, including customer service supports in the 3 new cities, technology experts in IT and AI.
 
•The timeline for this expansion is within the next 36 months.


##################################################
[{'Section': 'Project Details', 'Sub-section': 'Project Summary', 'Breakdown': '', 'Value': "The project for GogoX involves expanding its business in three cities in China (Beijing, Shanghai, and Shenzhen), making technology investments such as implem

{'section': 'Project Details',
 'output': "The project for GogoX involves expanding its business in three cities in China: Beijing, Shanghai, and Shenzhen. The objective is to support GogoX's expansion plans and meet its growth targets within the next 36 months.\nThe scope and scale of the project include:\n- Expansion plans in 3 cities in China: Beijing, Shanghai, and Shenzhen\n- Technology investments, including AI assistant for drivers and AI-based matching between drivers and customers\n- Working capital needs to expand the staff by 300 permanent employees, including customer service support in the 3 new cities and technology experts in IT and AI\nThe timeline for the expansion is within the next 36 months.\nThe necessary resources for the project's success include:\n- Financial resources: GogoX is requesting $10 million to support its expansion plans, including business expansion, technology investments, and working capital needs.\n- Personnel resources: GogoX plans to expand its 

In [5]:
from common.prompts import PROPOSAL_TEMPLATE_FORMATTING_PROMPT 

In [6]:
section_name_list = [
"Executive Summary", 
"Client Request", 
"Shareholders and Group Structure", 
"Project Details", 
"Industry / Section Analysis", 
"Management", 
"Financial Information of the Borrower", 
"Other Banking Facilities", 
"Opinion of the Relationship Manager", 
"Summary of Recommendation"
]

output_list = []
rm_fill_list = []

# for section_name in section_name_list:
#     output_json = run_first_gen(section=section_name, 
#               rm_note_txt=RM_text, 
#               client=client, 
#               deployment_name=DEPLOYMENT_NAME, 
#               openai_api_version=OPENAI_API_VERSION, 
#               openai_api_base=OPENAI_API_BASE)
#     output_list.append(output_json["output"])
#     rm_fill_list.append(output_json["RM_fill"])
    

In [7]:
output_list


[]

In [8]:
rm_fill_list

[]

In [9]:
# Re-generate function
def regen(section_name, previous_paragraph, rm_instruction, client="", deployment_name=DEPLOYMENT_NAME, openai_api_version=OPENAI_API_VERSION, openai_api_base=OPENAI_API_BASE):

    # set temperature as 0 with exception cases
    temperature = 0.5 if section_name == "Industry / Section Analysis" else 0

    # set up openai environment - Jay
    llm_proposal = AzureChatOpenAI(deployment_name=deployment_name, temperature=temperature,
                            openai_api_version=openai_api_version, openai_api_base=openai_api_base)
    chain = LLMChain(
        llm=llm_proposal,
        prompt=PromptTemplate(template=prompts.PROPOSAL_TEMPLATE_REGEN, input_variables=["previous_paragraph", "rm_instruction"]),
        output_key="re_gen_paragraph", verbose=True
    )

    review_chain = LLMChain(llm=llm_proposal
                            , prompt=PromptTemplate(template=prompts.PROPOSAL_TEMPLATE_REGEN_REVIEW, input_variables=["re_gen_paragraph"])
                            , output_key="reviewed",verbose=True)

    checking_formatting_chain = LLMChain(llm=llm_proposal
                                , prompt=PromptTemplate(template=prompts.PROPOSAL_TEMPLATE_FORMATTING_PROMPT, input_variables=["reviewed"])
                                , output_key="reviewed_2",verbose=True)

    if section_name in ["Industry / Section Analysis", "Summary of Recommendation"]:
        overall_chain = SequentialChain(chains=[chain], 
                                        input_variables=["previous_paragraph", "rm_instruction"],
                                        # Here we return multiple variables
                                        output_variables=["re_gen_paragraph"],
                                        verbose=True)
    else:
        overall_chain = SequentialChain(chains=[chain, review_chain, checking_formatting_chain], 
                                        input_variables=["previous_paragraph", "rm_instruction"],
                                        # Here we return multiple variables
                                        output_variables=["reviewed_2"],
                                        verbose=True)

    drafted_text = overall_chain({"previous_paragraph": previous_paragraph, "rm_instruction":rm_instruction})
    if section_name in ["Industry / Section Analysis","Summary of Recommendation"]:
        drafted_text = drafted_text["re_gen_paragraph"]
    else:
        drafted_text = drafted_text["reviewed_2"]
    drafted_text2 = drafted_text.replace("Based on the given information, ", "").replace("It is mentioned that ", "")

    #All capital letters for first letter in sentences
    formatter = re.compile(r'(?<=[\.\?!]\s)(\w+)')
    drafted_text2 = formatter.sub(lambda m: m.group().capitalize(), drafted_text2)

    # Capitalize the first character of the text
    drafted_text2 = drafted_text2[0].capitalize() + drafted_text2[1:]

    rm_fill_values = []

    # Loop the RM notes missing information into the generate part
    #for i in rm_text_variable:
    #    rm_fill_values.append(i)

    lines = drafted_text.split("\n")

    for i, line in enumerate(lines):
        matches = re.findall(r"\[RM (.+?)\]\.?", line)  # Find all [RM ...] followed by optional dot
        for match in matches:
            rm_fill = match + "\n"
            rm_fill_values.append(rm_fill)
        
        # remove all the RM requests and the optional following dots from the line
        line = re.sub(r"\[RM .+?\]\.?", "", line)
        lines[i] = line

    # Rejoin the lines into a single string without RM requests
    drafted_text2 = "\n".join(lines)
    drafted_text3 = clean_generated_text(drafted_text2, client, section_name)
    final_rm_fill_text = generate_rm_fill(rm_fill_values, client)
    #print(rm_fill_values)

    output_json = {
        "section": section_name,
        "output": drafted_text3,
        "RM_fill" : final_rm_fill_text,
    }

    #output the result
    return output_json

In [10]:
client = "GogoX"

previous_paragraph = ""

rm_instruction = "During the Reporting Period, the Group did not have any bank loans and other borrowings."

#section_name ="Financial Information of the Borrower"

section_name = "Other Banking Facilities"

client = ""

regen(
    section_name, previous_paragraph, rm_instruction
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        # Instruction
       
        - Given the following extracted parts under ----Previous Paragraph---- and ----RM Instructions----, create a final summary based on these parts **ONLY**.
        - Take note of the content inside the square bracket about missing information, if the information is still missing, request these missing information using this format: '[RM Please provide further information on XXX]' at the end of your answer. 
        
        ## About your output format:
        - Remove any bullet forms from the input paragraph
        - Remove any sentences stating or implying that information is missing or not provided, such as 'However, further information is needed to provide a more comprehensive summary of the project details.' or 'Additionally, No specific information is provided about the proposed loan facility.' or "Additionally, no information is provided re


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

        # Instruction

        Given the following extracted parts under ----Input Paragraph----, create a final summary based on the following output format. 


        ## About your output format:
        -**Must** not change or edit or remove the sentence inside the square bracket []
        -**Must** remove the sentences that contains the phrase or meaning of "is not mentioned" or "is not provided" or "are not mentioned" or "are not provided"
        -**Must** remove the sentences that contains the phrase or meaning of "further information is needed"
        -**Must** remove any sentences that contains the phrase "are mentioned" or "is mentioned"

        ----Input Paragraph----
        During the Reporting Period, the Group did not have any bank loans and other borrowings.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

        To complete this task, you need to review

{'section': 'Other Banking Facilities',
 'output': 'The Group did not have any bank loans and other borrowings during the Reporting Period.',
 'RM_fill': ''}

In [11]:
prompts.PROPOSAL_TEMPLATE_REGEN

'\n        # Instruction\n       \n        - Given the following extracted parts under ----Previous Paragraph---- and ----RM Instructions----, create a final summary based on these parts **ONLY**.\n        - Take note of the content inside the square bracket about missing information, if the information is still missing, request these missing information using this format: \'[RM Please provide further information on XXX]\' at the end of your answer. \n        \n        ## About your output format:\n        - Remove any bullet forms from the input paragraph\n        - Remove any sentences stating or implying that information is missing or not provided, such as \'However, further information is needed to provide a more comprehensive summary of the project details.\' or \'Additionally, No specific information is provided about the proposed loan facility.\' or "Additionally, no information is provided regarding the proposed loan facility.", these must be removed entirely from your output.\

In [12]:
client = "GogoX"


section_name ="Financial Information of the Borrower"

#section_name = "Summary of Recommendation"

RM_text="""

•Request $10 million to support its expansion plans        
•Repayment plan: Regular principal and interest payments over a 3 years term
 
•The Expansion plans include 3 areas:
•Expanding the business in 3 cities in China (Beijing, Shanghai and Shenzhen)
•Technology investments (AI assistant fo drivers and AI-based matching among drivers and customers)
•Working capital needs: expanding 300 more permanent staff, including customer service supports in the 3 new cities, technology experts in IT and AI.
 
•The timeline for this expansion is within the next 36 months.

"""

run_first_gen(section=section_name, 
              rm_note_txt=RM_text, 
              client=client, 
              deployment_name=DEPLOYMENT_NAME, 
              openai_api_version=OPENAI_API_VERSION, 
              openai_api_base=OPENAI_API_BASE)

##################################################
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
rm_note_txt printing: 

•Request $10 million to support its expansion plans        
•Repayment plan: Regular principal and interest payments over a 3 years term
 
•The Expansion plans include 3 areas:
•Expanding the business in 3 cities in China (Beijing, Shanghai and Shenzhen)
•Technology investments (AI assistant fo drivers and AI-based matching among drivers and customers)
•Working capital needs: expanding 300 more permanent staff, including customer service supports in the 3 new cities, technology experts in IT and AI.
 
•The timeline for this expansion is within the next 36 months.


##################################################
[{'Section': 'Financial Information of the Borrower', 'Sub-section': 'Financial Statements', 'Breakdown': '', 'Value': "RM Please provide further information on the borrower's audited financial statements for the past 2 to 3 years (Refer to the questi

{'section': 'Financial Information of the Borrower',
 'output': '',
 'RM_fill': "Please provide further information on Please provide information on the borrower's audited financial statements for the past 2 to 3 years, revenue sources and cost structure, financial performance, financial trends and future prospects, significant events or facts, and the equity to debt ratio, net income, and ROE of the borrower."}

In [13]:
section_name = "Other Banking Facilities"

web_extract_RM(section_name, RM_text, client)

##################################################
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
rm_note_txt printing: 

•Request $10 million to support its expansion plans        
•Repayment plan: Regular principal and interest payments over a 3 years term
 
•The Expansion plans include 3 areas:
•Expanding the business in 3 cities in China (Beijing, Shanghai and Shenzhen)
•Technology investments (AI assistant fo drivers and AI-based matching among drivers and customers)
•Working capital needs: expanding 300 more permanent staff, including customer service supports in the 3 new cities, technology experts in IT and AI.
 
•The timeline for this expansion is within the next 36 months.


##################################################
[{'Section': 'Other Banking Facilities', 'Sub-section': 'Existing Loans', 'Breakdown': '', 'Value': "Based on the information provided , there is no mention of the borrower's existing credit facilities from other banks or financial institutions. There

([{'Section': 'Other Banking Facilities',
   'Sub-section': 'Existing Loans',
   'Breakdown': '',
   'Value': "Based on the information provided , there is no mention of the borrower's existing credit facilities from other banks or financial institutions. Therefore, it is not possible to extract any details regarding the name of the lending institution, type of facility, outstanding balance, interest rate, maturity date, or any collateral or guarantees associated with each facility. \nRM Please provide further information on the borrower's existing credit facilities from other banks or financial institutions.",
   'Question': "Please extract the list of the borrower's existing credit facilities from other banks or financial institutions, including details such as the name of the lending institution, type of facility, outstanding balance, interest rate, maturity date and any collateral or guarantees associated with each facility.",
   'Example': "Currently, the company has an existing l